<a href="https://colab.research.google.com/github/sabrybenamor/Machine-Learning-with-Python/blob/master/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers

print(tf.__version__)

2.20.0-dev20250604


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-06-10 13:49:09--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-06-10 13:49:10 (9.79 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-06-10 13:49:10--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

TSV_COLUMN_NAMES = ["Nature", "Message core"]
train_df = pd.read_csv(train_file_path, names=TSV_COLUMN_NAMES, sep='\t')
test_df = pd.read_csv(test_file_path, names=TSV_COLUMN_NAMES, sep='\t')


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df["Nature"].values)
y_test = label_encoder.transform(test_df["Nature"].values)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_df["Message core"].values).toarray()
X_test = vectorizer.transform(test_df["Message core"].values).toarray()

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8579 - loss: 0.5569 - val_accuracy: 0.8636 - val_loss: 0.3812
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9342 - loss: 0.2282 - val_accuracy: 0.9737 - val_loss: 0.1397
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9877 - loss: 0.0762 - val_accuracy: 0.9809 - val_loss: 0.0804
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9931 - loss: 0.0352 - val_accuracy: 0.9821 - val_loss: 0.0662
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9970 - loss: 0.0197 - val_accuracy: 0.9844 - val_loss: 0.0619
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9988 - loss: 0.0119 - val_accuracy: 0.9833 - val_loss: 0.0598
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9994 - loss: 0.0079 - val_accuracy: 0.9833 - val_loss: 0.0603
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9994 - loss: 0.0054 - val_accuracy: 0

In [ ]:
def prepare_input(messages):
    """
    Transforms a list or array of raw text messages into numerical vectors
    using the globally available fitted TF-IDF vectorizer.

    Args:
        messages (list or np.array): The raw SMS messages (strings).

    Returns:
        np.array: The TF-IDF-transformed input suitable for model.predict or model.evaluate.
    """
    return vectorizer.transform(messages).toarray()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = []
    pred_input = prepare_input([pred_text])
    predictions = model.predict(pred_input)
    # Cast to regular Python float
    result = float(predictions[0][0] if predictions.ndim == 2 else predictions[0])
    if result < 0.5:
        prediction.append(result)
        prediction.append("ham")
    else:
        prediction.append(result)
        prediction.append("spam")
    return prediction

# Example usage:
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[0.000592277676332742, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
You passed the challenge. Great job!
